In [57]:
from tensorflow import keras
import preprocess_tools as pts
import pandas as pd
#!pip install dataframe-image
import dataframe_image as dfi
import pickle
def pipeline(data, path_to_model, multilabelbinarizer):
    """
    load model cand compute prediction
    """
    data["Body"]  = data["Title"] + data["Body"]
    data = data.drop(["Title", "CreationDate"], axis=1)
    print("input data shape\n",data.head())
    
    
    data['Body'] = data['Body'].apply(pts.preprocess) 
    
    print("...preprocessed")
    path_to_tokenizer = "./Data_preparation_notebook/top11_tokenizer.pickle"
    pickle_in = open(path_to_tokenizer,"rb")
    tokenizer = pickle.load(pickle_in)
    print("...word to sequence")
    X_unknown = tokenizer.texts_to_sequences(data["Body"])
    X_unknown = keras.preprocessing.sequence.pad_sequences(X_unknown)
    
    
    #path_to_model = "./SavedModel/RNN_V2_nb_epochs_30/"
    model = keras.models.load_model(path_to_model)
    print("... model sucessfully loaded")
    y = model.predict(X_unknown)
    predictions = np.round(y)

    for i in range(predictions.shape[0]):
        arr = predictions[i]
        if sum(arr) == 0:
            predictions[i,np.argmax(y[i])] =1
    print(predictions.shape)
    predictions_df = pd.DataFrame(predictions,
                   columns=['<android>', '<c#>', '<c++>', '<css>', '<html>', '<java>', '<javascript>',
                    '<python>', '<r>', '<sql>'])
    
    dfi.export(predictions_df, 'dataframe.png')
    return predictions_df


In [58]:
import pandas as pd
pickle_in = open("./Data_preparation_notebook/mlb.pickle","rb")
mlb = pickle.load(pickle_in)

unknown_data = pd.read_csv("../QueryResults/QueryResults/QueryResultsAndroid.csv").iloc[:10]

predictions = pipeline(unknown_data, "./SavedModel/RNN_V2_RNN_V2_14/", mlb)


C:\Users\Arnaud\anaconda3\envs\tf_2.6\lib\site-packages\sklearn\base.py:324: UserWarning: Trying to unpickle estimator MultiLabelBinarizer from version 0.22.2.post1 when using version 1.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(


input data shape
          Id                                               Body  \
0  36920138  json Array Parsing using Gson<p>i have the fol...   
1  11237936  Mobile Web - Disable long-touch/taphold text s...   
2  51964390  Get camera zoomlevel from googlemaps in Kotlin...   
3  25812055  Android Wear BoxInsetLayout isRound always ret...   
4  17728266  how to call a user defined method in a custom ...   

                                        Tags  
0             <android><json><parsing><gson>  
1  <android><iphone><css><mobile><highlight>  
2             <android><google-maps><kotlin>  
3             <android><android-ui><wear-os>  
4                                  <android>  
...preprocessed
...word to sequence
... model sucessfully loaded
(10, 10)


In [60]:
print(predictions[:50])

   <android>  <c#>  <c++>  <css>  <html>  <java>  <javascript>  <python>  <r>  \
0        1.0   0.0    0.0    0.0     0.0     1.0           0.0       0.0  0.0   
1        0.0   0.0    0.0    0.0     0.0     0.0           1.0       0.0  0.0   
2        1.0   0.0    0.0    0.0     0.0     0.0           0.0       0.0  0.0   
3        0.0   1.0    0.0    0.0     0.0     0.0           0.0       0.0  0.0   
4        1.0   1.0    0.0    0.0     0.0     0.0           0.0       0.0  0.0   
5        1.0   0.0    0.0    0.0     0.0     0.0           0.0       0.0  0.0   
6        1.0   0.0    0.0    0.0     0.0     0.0           0.0       0.0  0.0   
7        1.0   0.0    0.0    0.0     0.0     0.0           0.0       0.0  0.0   
8        1.0   0.0    0.0    0.0     0.0     0.0           0.0       0.0  0.0   
9        0.0   0.0    1.0    0.0     0.0     0.0           0.0       0.0  0.0   

   <sql>  
0    0.0  
1    0.0  
2    0.0  
3    0.0  
4    0.0  
5    0.0  
6    0.0  
7    0.0  
8    0.0 

In [39]:
unknown_data

,Id,Body,Title,Tags,CreationDate
0,36920138,json Array Parsing using Gson<p>i have the fol...,json Array Parsing using Gson,<android><json><parsing><gson>,2016-04-28 16:14:00
1,11237936,Mobile Web - Disable long-touch/taphold text s...,Mobile Web - Disable long-touch/taphold text s...,<android><iphone><css><mobile><highlight>,2012-06-28 04:07:11
2,51964390,Get camera zoomlevel from googlemaps in Kotlin...,Get camera zoomlevel from googlemaps in Kotlin,<android><google-maps><kotlin>,2018-08-22 10:01:53
3,25812055,Android Wear BoxInsetLayout isRound always ret...,Android Wear BoxInsetLayout isRound always ret...,<android><android-ui><wear-os>,2014-09-12 15:45:16
4,17728266,how to call a user defined method in a custom ...,how to call a user defined method in a custom ...,<android>,2013-07-18 15:50:14
5,17728282,osmdroid - java.lang.UnsupportedOperationExcep...,osmdroid - java.lang.UnsupportedOperationExcep...,<android><android-studio><osmdroid><openstreet...,2013-07-18 15:50:50
6,63781419,Draw pixel art in Jetpack Compose<p>I need to ...,Draw pixel art in Jetpack Compose,<android><kotlin><android-jetpack-compose>,2020-09-07 16:44:52
7,3967694,Android get time format<p>Is there a nice way ...,Android get time format,<android><datetime><format>,2010-10-19 11:07:50
8,54143856,How to prevent double items but also prevent e...,How to prevent double items but also prevent e...,<android>,2019-01-11 09:40:51
9,54143881,error: attribute behaviour_peekHeight not foun...,error: attribute behaviour_peekHeight not found,<android><android-studio><bottom-sheet>,2019-01-11 09:42:17
